In [1]:
import pandas as pd
import time
import geopandas
from glob import glob
from shapely.ops import cascaded_union
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
bids = pd.read_csv("data/all_bids.csv")[["round", "bidder", "tier", "latency", "census_id"]]
bids["state"] = bids["census_id"]//10000000000
bids = bids.groupby(["bidder", "census_id"]).agg({'round': 'max', 'state':'first'}).reset_index()
bids

,bidder,census_id,round,state
0,"1stel, Inc.",10010207001,9,1
1,"1stel, Inc.",50014801001,14,5
2,"1stel, Inc.",50014801002,14,5
3,"1stel, Inc.",50014802001,14,5
4,"1stel, Inc.",50014802002,14,5
...,...,...,...,...
424226,yondoo Broadband LLC,291138101002,17,29
424227,yondoo Broadband LLC,291138101003,18,29
424228,yondoo Broadband LLC,292214602003,12,29
424229,yondoo Broadband LLC,292214603003,18,29


In [3]:
name_map = {
    "CCO Holdings, LLC":"Charter Communications Inc",
    "Windstream Services LLC, Debtor-In-Possession":"Windstream Holdings, Inc.",
    "Connect Everyone LLC":"Starry, Inc.",
    "Frontier Communications Corporation, DIP":"Frontier Communications Corporation",
    "California Internet, L.P. dba GeoLinks":"GeoLinks",
    "Etheric Communications LLC":"Etheric Networks, Inc.",
    "Consolidated  Communications, Inc.":"Consolidated Communications, Inc.",
    "Frontier Communications Northwest, LLC":"Ziply Fiber",
    "Citynet West Virginia, LLC":"Citynet, LLC", 
    "Computer 5, Inc. d/b/a LocalTel Communications": "LocalTel Communications",
    "Armstrong Telephone Company - Northern Division": "Armstrong Holdings, Inc.",
    "Commnet Wireless, LLC": "ATN International, Inc.",
    "Mediacom Communications Corporation": "Mediacom Communications Corp.",
    "Continental Divide Electric Cooperative": "Continental Divide Electric Cooperative, Inc.",
    "South Arkansas Telephone Company": "South Arkansas Telephone Co.",
    "Digital Connections Inc. dba PRODIGI": "Digital Connections, Inc.",
    "Shenandoah Cable Television, LLC": "Shenandoah Telecommunications Company",
    "Direct Communications Rockland, Inc": "Direct Communications Rockland, Inc.",
    "Pine Belt Communications, Inc.":"Pine Belt Communications Co. Inc.",
    "Savage Communications":"Savage Communications Inc.",
    "Hamilton County Telephone Co-op":"Hamilton County Telephone Co-Op",
    "Hotwire Communications, Ltd":"Hotwire Communications Ltd.",
    "Altice USA, Inc.":"Altice",
    "TruVista Communications, Inc.":"TruVista Communications of Georgia, LLC",
    "Hawaii Dialogix Telecom LLC":"Hawaii Dialogix Telecom LLC",
    "Safelink Internet LLC":"Safelink Internet",
    "American Heartland":"Farmers Telephone Company",
    "Cellular Services LLC.":"Cellular Services LLC",
    "Scott County Telephone Cooperative, Inc.":"Scott County",
    "Micrologic Inc.":"Micrologic, Inc.",
    "Pioneer Wireless, Inc":"Pioneer Wireless, Inc.",
    "St. John Telco":"St. John Telephone, Inc.",
    "One Ring Networks, Inc":"One Ring Networks, Inc.",
    "Emery Telephone dba Emery Telcom":"Emery Telcom",
    "XIT Telecommunication & Technology":"XIT Rural Telephone Cooperative, Inc.",
    "Somerset Telephone Co., Inc.":"SOMERSET TELEPHONE COMPANY",
    "Siuslaw Broadband, LLC dba Hyak Technologies":"Siuslaw Broadband, LLC",
    "Minnesota Connections c/o Consolidated Tel Company":"Consolidated Telephone Company",
    "DoCoMo Pacific, Inc.":"Docomo Pacific",
    "Roseau Electric Cooperative, Inc.":"Roseau Electric Cooperative",
    "Custer Telephone Cooperative, Inc.":"Custer Telephone Cooperative Inc.",
    "Lakeland Communications Group, LLC":"Lakeland Communications, Inc.",
    "LigTel Communications, Inc.":"Ligtel Communications",
    "Hamilton Long Distance Company":"Hamilton.net, Inc.",
    "Allen's T.V. Cable Service, Inc.":"Allen's TV Cable Service, Inc.",
    "W. T. Services, Inc.":"W.T. Services, Inc.",
    "Heart of the Catskills Comm. Inc., dba MTC Cable":"MTC Cable",
    "Citizens Vermont Acquisition Corporation":"Citizens Telephone Company",
    "Nova Cablevision, Inc":"Nova Cablevision, Inc.",
    "yondoo Broadband LLC":"yondoo Broadband, LLC",
    "Baraga Telephone Company":"Baraga Telephone Company Inc.",
    "Miles Communications LLC":"Miles Communications, Inc.",
    "PVT NetWorks, Inc.":"Penasco Valley Telephone Cooperative, Inc.",
    "Yucca Telecommunications Systems, Inc.":"Yucca Telecom",
    "H&B Communication's, Inc.":"H&B Enterprises Inc.",
    "MARQUETTE-ADAMS TELEPHONE COOPERATIVE, INC.":"Marquette-Adams Telephone Cooperative, Inc.",
    "Pinpoint Bidding Coalition":"Pinpoint Holdings, Inc.",
    "Computer Techniques, Inc. dba CTI Fiber":"CTI Fiber",
    "Barry Technology Services, LLC":"Barry Technology Services",
    "Bloosurf, LLC":"Bloosurf",
    "St Paul Cooperative Telephone Association":"St Paul Coop Telephone Assoc",
    "Farmers Mutual Cooperative Telephone Company":"FARMERS MUTUAL TELEPHONE COMPANY",
    "NTS Communications, LLC":"NTS, Inc.",
    "Unified Communications Inc.":"Unified Communications, Inc.",
    "Peoples Communication, LLC.":"Peoples Communication, Inc.",
    "Wood County Telephone Company d/b/a Solarus":"Solarus",
    "Comcell Inc.":"Comcell, Inc.",
    "Taylor Telephone Coop., Inc. dba Taylor Telecom":"Taylor Telephone Cooperative, Inc.",
    "Plateau Telecommunications, Incorporated":"Plateau",
    "AMA Communications, LLC":"AMA Communicaitons, L.L.C.",
    "AirCell, Inc.":"AirCell",
    "Bays-ET Highspeed LLC":"Bays-ET Highspeed Internet LLC",
    "Big Bend Telecom LTD":"Big Bend Telephone Company, Inc.",
    "Blue Ridge Cable Technologies, Inc.":"Blue Ridge Cable Technologies, Inc.",
    "Cass Cable TV, Inc.":"Cass Cable TV, Inc.",
    "City of Wilson GreenLight":"City of Wilson",
    "Climax Telephone Company dba CTS Telecom, Inc.":"Climax Telephone Company",
    "Cumberland Telephone Company, Inc":"CUMBERLAND TELEPHONE COMPANY",
    "Cumby Telephone Coooertive, Inc.":"Cumby Telephone Coop., Inc.",
    "Data Stream Mobile Technoligies Inc.":"Data Stream",
    "Get Wireless Inc":"Getwireless.net",
    "Hartington Telecommunications Co., Inc,":"Hartington Telecommunications Co., Inc.",
    "Hilliary Communications Consortium":"Hilliary Communications, LLC",
    "Hillsboro Telephone Company,  Inc.":"Hillsboro Telephone Company",
    "Home Telephone ILEC, LLC":"Home Telephone ILEC, LLC d/b/a Home Telecom",
    "Hood Canal Telephone Co., Inc":"Hood Canal Communications",
    "Kingdom Telecommunications, Inc.":"Kingdom Telecommunications Inc",
    "Lincolnville Communications":"Lincolnville Telephone Company",
    "Local Internet Service Company, Inc.":"Local Internet Service Company",
    "Martell Enterprise  Inc.":"Martell Enterprises, Inc.",
    "Massena Telephone Company, Inc.":"Massena Telephone Company",
    "Moundridge Telephone Company":"Moundridge Telephone Co.",
    "MyServer.org, Inc dba San Diego Broadband":"San Diego Broadband",
    "Newbreak Management, LLC":"Newbreak Communications",
    "NexGenAccess Incorporated":"NexGenAccess",
    "Northwoods Communication Technologies, LLC":"Northwoodsconnect",
    "One Point Technologies Inc":"One Point Technologies Inc.",
    "Panhandle Telecommunication Systems, Inc.":"Panhandle Telephone Cooperative, Inc.",
    "Pathwayz Communications Inc":"Pathwayz Communications, Inc.",
    "RONAN TELEPHONE COMPANY":"Ronan Telephone Co",
    "Rainbow Communications LLC":"Rainbow Telecommunications Association, Inc.",
    "SOUTHWEST ARKANSAS TELEPHONE COOPERATIVE, INC.":"Southwest Arkansas Telephone Cooperative, Inc.",
    "South Central Wireless, Inc.":"South Central Wireless Inc.",
    "Southern Montana Telephone Company (SMTC)":"Southern Montana Telephone Company",
    "Steelville Telephone Exchange Inc.":"Steelville Telephone Exchange Inc",
    "Tekstar Communications, Inc. dba Arvig":"Arvig Enterprises, Inc.",
    "Upsala Cooperative Telephone Association dba Sytek":"UPSALA COOPERATIVE TELEPHONE ASSOCIATION",
    "Valley Telephone Cooperative, Inc":"Valley Telephone Cooperative, Inc.",
    "Velocity.Net Communications, Inc.":"Velocity Communications, Inc.",
    "Woodstock Telephone Co.":"Woodstock Telephone Company",
    "ZIRKEL Wireless, LLC":"Zirkel Wireless",
    "coon valley cooperative telephone association inc.":"Coon Valley Co-op Telephone Association, Inc.",
}

bids["bidder"] = bids["bidder"].map(lambda x: name_map[x] if x in name_map else x)
bids

,bidder,census_id,round,state
0,"1stel, Inc.",10010207001,9,1
1,"1stel, Inc.",50014801001,14,5
2,"1stel, Inc.",50014801002,14,5
3,"1stel, Inc.",50014802001,14,5
4,"1stel, Inc.",50014802002,14,5
...,...,...,...,...
424226,"yondoo Broadband, LLC",291138101002,17,29
424227,"yondoo Broadband, LLC",291138101003,18,29
424228,"yondoo Broadband, LLC",292214602003,12,29
424229,"yondoo Broadband, LLC",292214603003,18,29


In [4]:
existing = pd.read_csv("data/existing_service.csv", encoding="latin")
existing["bg"] = existing["BlockCode"]//1000
existing  = existing.loc[(existing["MaxAdDown"]>=25) & (existing["MaxAdUp"]>=3) & (existing["TechCode"]!=60)]
existing["state"] = existing["bg"]//10000000000
existing['names'] = [frozenset(x) for x in zip(existing.ProviderName, existing.HoldingCompanyName, existing.DBAName, existing.HocoFinal)]
existing

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,bg,state,names
0,1,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501002019,70,1,40.0,10.0,1,410139501002,41,"(Webformix Company, Webformix)"
1,2,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139503002023,70,1,40.0,10.0,1,410139503002,41,"(Webformix Company, Webformix)"
2,3,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139502002027,70,1,40.0,10.0,1,410139502002,41,"(Webformix Company, Webformix)"
3,4,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501003004,70,1,40.0,10.0,1,410139501003,41,"(Webformix Company, Webformix)"
4,5,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139504001109,70,1,40.0,10.0,1,410139504001,41,"(Webformix Company, Webformix)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42151570,42151571,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003006,70,1,30.0,10.0,1,720054006003,72,"(OSNET Wireless, Osnet Wireless Corp., OSNET W..."
42151571,42151572,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003005,70,1,30.0,10.0,1,720054006003,72,"(OSNET Wireless, Osnet Wireless Corp., OSNET W..."
42151572,42151573,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003004,70,1,30.0,10.0,1,720054006003,72,"(OSNET Wireless, Osnet Wireless Corp., OSNET W..."
42151573,42151574,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003003,70,1,30.0,10.0,1,720054006003,72,"(OSNET Wireless, Osnet Wireless Corp., OSNET W..."


In [5]:
eligible = pd.read_csv("data/auction904_updated_block_cbg.csv")
winners = pd.read_csv("data/all_assigned_census_blocks.csv")[["bidder","block_id","state","census_id"]]
winners = winners.groupby(['bidder','census_id'], as_index=False).agg({"block_id":list, "state":"first"})
winners["bidder"] = winners["bidder"].map(lambda x: name_map[x] if x in name_map else x)
winners

,bidder,census_id,block_id,state
0,4-Corners Consortium,350319437004,"[350319437004005, 350319437004010, 35031943700...",NM
1,4-Corners Consortium,350319438004,"[350319438004000, 350319438004003, 35031943800...",NM
2,AB Indiana LLC,120210112021,"[120210112021071, 120210112021072, 12021011202...",FL
3,ACT,281159504002,"[281159504002011, 281159504002012, 28115950400...",MS
4,ACT,281159505001,"[281159505001024, 281159505001026, 28115950500...",MS
...,...,...,...,...
57167,Yucca Telecom,350410003002,"[350410003002004, 350410003002063]",NM
57168,Yucca Telecom,350410003003,[350410003003024],NM
57169,"Zito West Holding, LLC",420879605002,"[420879605002001, 420879605002004, 42087960500...",PA
57170,"yondoo Broadband, LLC",291138101002,"[291138101002030, 291138101002058, 29113810100...",MO


In [6]:
shapefiles = glob("data/all_data/*.shp")
county_shapes = []
for file in  shapefiles:
    county_shapes.append(geopandas.read_file(file))

county_shapes = pd.concat(county_shapes)[["STATEFP", "COUNTYFP", "TRACTCE", "BLKGRPCE", "GEOID", "geometry"]]
county_shapes["GEOID"] = pd.to_numeric(county_shapes["GEOID"])
county_shapes

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,geometry
0,05,143,010503,2,51430105032,"POLYGON ((-94.21915 36.17593, -94.21718 36.175..."
1,05,143,010504,1,51430105041,"POLYGON ((-94.33762 35.97984, -94.33724 35.980..."
2,05,143,010504,5,51430105045,"POLYGON ((-94.43864 35.95907, -94.43852 35.959..."
3,05,143,010506,1,51430105061,"POLYGON ((-94.24059 36.04502, -94.24058 36.045..."
4,05,135,470200,7,51354702007,"POLYGON ((-91.54745 36.26023, -91.54734 36.260..."
...,...,...,...,...,...,...
3921,24,003,740601,4,240037406014,"POLYGON ((-76.77774 39.11425, -76.77746 39.114..."
3922,24,510,160500,1,245101605001,"POLYGON ((-76.66090 39.30614, -76.66017 39.306..."
3923,24,510,160500,2,245101605002,"POLYGON ((-76.65502 39.29848, -76.65464 39.298..."
3924,24,510,160500,3,245101605003,"POLYGON ((-76.65941 39.29463, -76.65892 39.294..."


In [7]:
try:
    state_shapes = pickle.load(open("state_shapes.pkl", "rb" ))
except FileNotFoundError:
    state_shapes = county_shapes.groupby("STATEFP")["geometry"].apply(list).reset_index(name="geometry")
    state_shapes["geometry"] = state_shapes["geometry"].apply(cascaded_union)
    state_shapes = geopandas.GeoDataFrame(state_shapes)
    for index, row in state_shapes.iterrows():
        neighbors = state_shapes[~state_shapes.geometry.disjoint(row['geometry'])]
        state_shapes.at[index, "neighbors"] = ", ".join(neighbors.STATEFP.tolist())
    with open('state_shapes.pkl', 'wb') as f:
        pickle.dump(state_shapes, f)

In [8]:
class BiddingBlockGroup:
    def __init__(self, bg, bidders, neighbors, winner, blocks):
        self.bg = bg
        self.bidders = bidders
        self.neighbors = neighbors
        self.winner = winner
        self.blocks = blocks
        
def get_bids_with_neighbors(bidding_block_group):
    out = defaultdict(list)
    for nbg, nexisting_names in bidding_block_group.neighbors.items():
        for nexisting_name in nexisting_names:
            for bidder in bidding_block_group.bidders:
                if bidder in nexisting_name:
                    out[bidder].append(nbg)
    return out
    

In [3]:
try:
    processed_bid_table = pickle.load(open("neighbors.pkl", "rb" ))
except FileNotFoundError:
    with open("neighbors.log", "w") as log:
        pass

    total_time = 0
    processed_bid_list = []
    for state_code in state_shapes["STATEFP"].tolist():
        print(state_code)
        start = time.time()
        state_bids = bids.loc[bids["state"]==int(state_code)]
        state_bids = state_bids.groupby("census_id").agg({"bidder":list, "round":list}).reset_index()
        state_bids = state_bids.merge(county_shapes, left_on="census_id", right_on="GEOID")

        neighbors = state_shapes.loc[state_shapes["STATEFP"] == state_code]["neighbors"].item().split(", ")
        neighbors = [int(n) for n in neighbors]
        state_existing = existing.loc[existing["state"].isin(neighbors)]
        state_existing = state_existing.groupby("bg")["names"].apply(set).reset_index(name="names")
        state_existing = state_existing.merge(county_shapes, left_on="bg", right_on="GEOID")
        state_existing = geopandas.GeoDataFrame(state_existing)

        for index, row in state_bids.iterrows():
            bidders = row["bidder"]
            rounds = dict(zip(bidders, row["round"]))
            bg = row.census_id

            neighbors = state_existing[~state_existing.geometry.disjoint(row['geometry'])]
            ids = neighbors.GEOID.tolist()
            list_set_set_names = neighbors.names.tolist()
            neighbors_dict = dict(zip(ids, list_set_set_names))
            
            winner_data = winners.loc[winners.census_id == bg]
            winner_bidder = winner_data.bidder.tolist()
            winner_blocks = winner_data.block_id.tolist()
            winner_bidder = winner_bidder[0] if len(winner_bidder)==1 else None
            winner_blocks = winner_blocks[0] if len(winner_blocks)==1 else []

            
            bids_with_neighbors = defaultdict(list)
            for nbg, nexisting_names in neighbors_dict.items():
                for nexisting_name in nexisting_names:
                    for bidder in bidders:
                        if bidder in nexisting_name:
                            bids_with_neighbors[bidder].append(nbg)
            
            for bidder in bidders:
                processed_bid_list.append([bidder,
                                          bg,
                                          rounds[bidder],
                                          neighbors_dict,
                                          bids_with_neighbors[bidder],
                                          bidder == winner_bidder,
                                          winner_blocks if bidder == winner_bidder else [],
                                          row.geometry])
            
        state_time = time.time()-start
        with open("neighbors.log", "a") as log:
            log.write(f"{state_code} {state_time}\n")
        total_time += state_time
        print(state_time)

    with open("neighbors.log", "a") as log:
        log.write(f"final time {total_time}")

    processed_bid_table = geopandas.GeoDataFrame(processed_bid_list,
                                                 columns=["bidder", "GEOID", "round", "all_neighbors",
                                                          "neighbors", "winner", "won_blocks", "geometry"])
    with open('neighbors.pkl', 'wb') as f:
        pickle.dump(processed_bid_table, f)

In [20]:
def neighbor_status(census_id, neighbors):
    neighbors_set = set(neighbors)
    if len(neighbors_set)==0:
        return 0
    elif census_id in neighbors:
        return 2
    else:
        return 1

In [21]:
processed_bid_table["neighbor_status"] = processed_bid_table.apply(lambda x: neighbor_status(x.GEOID, x.neighbors), axis=1)

In [53]:
processed_bid_table["num_blocks_won"] = processed_bid_table.won_blocks.apply(len)
processed_bid_table["neighbor_bgs"] = processed_bid_table.neighbors.apply(len)
processed_bid_table["all_neighbor_bgs"] = processed_bid_table.all_neighbors.apply(len)
agg = processed_bid_table.groupby("bidder").agg({"GEOID":len, "round":"mean", "num_blocks_won":sum,"neighbor_bgs":"mean","all_neighbor_bgs":"mean", "winner":sum})
agg = agg.rename(columns={'GEOID': 'num bids', 'round': 'average final round number', 'num_blocks_won': 'total blocks won', 'neighbor_bgs': 'average neighbors per bid', 'all_neighbor_bgs': 'average any neighbors per bid', "winner":"num wins"}).reset_index()
existing = processed_bid_table.loc[processed_bid_table.neighbor_status == 2].groupby("bidder").agg({"GEOID":len, "round":"mean", "num_blocks_won":sum,"neighbor_bgs":"mean","all_neighbor_bgs":"mean", "winner":sum})
existing = existing.rename(columns={'GEOID': 'existing bg bids', 'round': 'existing average final round number', 'num_blocks_won': 'existing blocks won', 'neighbor_bgs': 'existing average neighbors per bid', 'all_neighbor_bgs': 'existing average any neighbors per bid', "winner":"existing num wins"}).reset_index()
wild_tract = processed_bid_table.loc[processed_bid_table.neighbor_status == 0].groupby("bidder").agg({"GEOID":len, "round":"mean", "num_blocks_won":sum,"neighbor_bgs":"mean","all_neighbor_bgs":"mean", "winner":sum})
wild_tract = wild_tract.rename(columns={'GEOID': 'wild tract bg bids', 'round': 'wild tract average final round number', 'num_blocks_won': 'wild tract blocks won', 'neighbor_bgs': 'wild tract average neighbors per bid', 'all_neighbor_bgs': 'wild tract average any neighbors per bid', "winner":"wild tract num wins"}).reset_index()
neighbor = processed_bid_table.loc[processed_bid_table.neighbor_status == 1].groupby("bidder").agg({"GEOID":len, "round":"mean", "num_blocks_won":sum,"neighbor_bgs":"mean","all_neighbor_bgs":"mean", "winner":sum})
neighbor = neighbor.rename(columns={'GEOID': 'neighbor bg bids', 'round': 'neighbor average final round number', 'num_blocks_won': 'neighbor blocks won', 'neighbor_bgs': 'neighbor neighbors per bid', 'all_neighbor_bgs': 'neighbor any neighbors per bid', "winner":"neighbor num wins"}).reset_index()
winner = processed_bid_table.loc[processed_bid_table.winner]
winner_agg = winner.groupby("bidder").agg({"round":"mean","neighbor_bgs":"mean","all_neighbor_bgs":"mean"})
winner_agg = winner_agg.rename(columns={'round': 'winner average final round number', 'neighbor_bgs': 'winner average neighbors per bid', 'all_neighbor_bgs': 'winner average any neighbors per bid'}).reset_index()
winner_existing = winner.loc[winner.neighbor_status == 2].groupby("bidder").agg({"round":"mean", "neighbor_bgs":"mean","all_neighbor_bgs":"mean"})
winner_existing = winner_existing.rename(columns={'round': 'existing winner average final round number', 'neighbor_bgs': 'existing winner average neighbors per bid', 'all_neighbor_bgs': 'existing winner average any neighbors per bid'}).reset_index()
winner_wild_tract = winner.loc[winner.neighbor_status == 0].groupby("bidder").agg({"round":"mean", "neighbor_bgs":"mean","all_neighbor_bgs":"mean"})
winner_wild_tract = winner_wild_tract.rename(columns={'round': 'wild tract winner average final round number', 'neighbor_bgs': 'wild tract winner average neighbors per bid', 'all_neighbor_bgs': 'wild tract winner average any neighbors per bid'}).reset_index()
winner_neighbor = winner.loc[winner.neighbor_status == 1].groupby("bidder").agg({"round":"mean","neighbor_bgs":"mean","all_neighbor_bgs":"mean"})
winner_neighbor = winner_neighbor.rename(columns={'round': 'neighbor winner average final round number', 'neighbor_bgs': 'neighbor winner neighbors per bid', 'all_neighbor_bgs': 'neighbor winner any neighbors per bid'}).reset_index()

final = winner_neighbor.merge(winner_wild_tract, how="outer").merge(winner_existing, how="outer").merge(winner_agg, how="outer").merge(neighbor, how="outer").merge(wild_tract, how="outer").merge(existing, how="outer").merge(agg, how="outer")
final = final.fillna(0)
final["wins/bid"] = final["num wins"]/final["num bids"]
final["blocks/win"] = final["total blocks won"]/final["num wins"]
final = final.fillna(0)

,bidder,neighbor winner average final round number,neighbor winner neighbors per bid,neighbor winner any neighbors per bid,wild tract winner average final round number,wild tract winner average neighbors per bid,wild tract winner average any neighbors per bid,existing winner average final round number,existing winner average neighbors per bid,existing winner average any neighbors per bid,...,existing average any neighbors per bid,existing num wins,num bids,average final round number,total blocks won,average neighbors per bid,average any neighbors per bid,num wins,wins/bid,blocks/win
0,AMG Technology Investment Group LLC,17.954023,2.034483,8.666667,17.475806,0.0,8.116935,18.011628,7.286822,8.271318,...,7.973580,258.0,19506,9.254435,54960,0.300984,7.793653,1833,0.093971,29.983633
1,"Albion Telephone Company, Inc.",13.000000,2.000000,12.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1,13.000000,29,2.000000,12.000000,1,1.000000,29.000000
2,"Allen's TV Cable Service, Inc.",16.000000,2.000000,6.500000,0.000000,0.0,0.000000,14.500000,4.875000,6.875000,...,8.000000,8.0,17,16.529412,57,5.000000,7.764706,10,0.588235,5.700000
3,Altice,15.600000,1.400000,5.800000,16.000000,0.0,5.500000,15.257143,5.485714,6.542857,...,7.049419,35.0,435,13.749425,425,4.685057,6.855172,44,0.101149,9.659091
4,Aptitude Internet LLC,19.000000,1.500000,8.500000,17.393939,0.0,7.939394,0.000000,0.000000,0.000000,...,8.500000,0.0,216,14.287037,1823,0.083333,7.060185,35,0.162037,52.085714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Sycamore Telephone Company,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,8.500000,0.0,4,13.750000,0,7.000000,8.500000,0,0.000000,NaN
311,UPSALA COOPERATIVE TELEPHONE ASSOCIATION,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,7.500000,0.0,4,16.000000,0,4.750000,7.500000,0,0.000000,NaN
312,West River Telecommunications Cooperative,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,7.500000,0.0,2,11.000000,0,7.000000,7.500000,0,0.000000,NaN
313,"Wilson Creek Communications, LLC",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,9.200000,0.0,5,14.000000,0,6.800000,9.200000,0,0.000000,NaN


In [56]:
final.to_csv("bidder_neighbor_percentage.csv", index=False)

In [57]:
processed_bid_table

,bidder,GEOID,round,all_neighbors,neighbors,winner,won_blocks,geometry,neighbor_status,num_blocks_won,neighbor_bgs,all_neighbor_bgs
0,"1stel, Inc.",10010207001,9,"{10010202002: {('Charter Communications Inc', ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
1,"Hughes Network Systems, LLC",10010207001,13,"{10010202002: {('Charter Communications Inc', ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
2,Simple Networks Systems LLC,10010207001,10,"{10010202002: {('Charter Communications Inc', ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
3,Space Exploration Technologies Corp.,10010207001,13,"{10010202002: {('Charter Communications Inc', ...",[],True,"[10010207001071, 10010207001073, 10010207001095]","POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,3,0,11
4,"Viasat, Inc.",10010207001,12,"{10010202002: {('Charter Communications Inc', ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...
424126,"Hughes Network Systems, LLC",560430002002,12,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424127,"Plains Internet, LLC",560430002002,10,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424128,Simple Networks Systems LLC,560430002002,1,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424129,Space Exploration Technologies Corp.,560430002002,13,{560039626001: {('Mountain West Technologies C...,[],True,"[560430002002314, 560430002002337, 56043000200...","POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,7,0,10
